In [74]:
from utilities_IGNOBEL import *
with open('credential.json','r') as f:
    cred = json.load(f)
cluster = MongoClient(cred['cred'])
db = cluster['Game']
collection = db['Players']
import glob 
import re

In [4]:
"""Usare queste rose se FG non ha quelle corrette, e.g. durante il mercato"""
#RoseSS = rose_complete()

#RoseFG = rose()

#labels = RoseFG.columns
#Rose = RoseSS[labels]

'Usare queste rose se FG non ha quelle corrette, e.g. durante il mercato'

In [4]:
#infortunati(giornata=1) #restituisce una singola lista con tutti gli infortunati di ogni squadra di serie A per la giornata indicata

In [2]:
#rose() #restituisce le rose attuali di ogni fantallenatore

In [3]:
#formazioni() #restituisce le formazioni schierate in ogni giornata (inclusa la panchina)

In [12]:
#per qualche strano motivo a volte da errore la prima volta che gira, 
#ma funziona le volte successive, potrebbe essere un caso
V = bonus_panchina(giornata=21) 
V
# restituisce i voti dei panchinari che non sono scesi in campo (link diverso temp)

,IGNORANZA EVERYWHERE,MAINZ NA GIOIA,XYZ,AS 800A,PALLA PAZZA,I DISEREDATI,SOROS FC,PDG 1908
Bonus Panchinari,0.5,0,0,1.5,0,1.5,0,0


In [35]:
G = goal_subiti(giornata=21)
G
# restituisce il totale dei gol subiti da ogni portiere sceso in campo

,IGNORANZA EVERYWHERE,MAINZ NA GIOIA,XYZ,AS 800A,PALLA PAZZA,I DISEREDATI,SOROS FC,PDG 1908
Goal subiti,1,2,2,0,0,0,0,0


In [36]:
M = modificatore(giornata=21) 
M
# restituisce il bonus modificatore difesa guadagnato da ogni squadra

,IGNORANZA EVERYWHERE,MAINZ NA GIOIA,XYZ,AS 800A,PALLA PAZZA,I DISEREDATI,SOROS FC,PDG 1908
Modificatore,0,0.5,0.5,0,1.5,1.5,0,1.5


In [37]:
C = cartellini(giornata=21)
C

,IGNORANZA EVERYWHERE,MAINZ NA GIOIA,XYZ,AS 800A,PALLA PAZZA,I DISEREDATI,SOROS FC,PDG 1908
C. gialli,0,3,2,1,1,0,0,0
C. rossi,1,0,0,0,0,0,0,0


In [2]:
CI = count_inf(infortunati(giornata = 28), rose()) #restituisce il numero di infortunati in ogni rosa
##NOTA: usa Rose definito sopra nel caso in cui FG non ha le rose corrette e quindi serve usare lo spreadsheet
CI

,XYZ,MAINZ NA GIOIA,I DISEREDATI,PALLA PAZZA,AS 800A,PDG 1908,SOROS FC,IGNORANZA EVERYWHERE
tot Infortunati,0,0,0,0,0,0,0,0


In [10]:
infortunati(giornata = 28)

[]

In [39]:
F = fantapunti_fatti(giornata=21)

In [41]:
S = fantapunti_subiti(giornata=21)

In [42]:
#tot = IGNOBEL_tot(giornata=6) # a causa di ritardi di connessione, potrebbe dare errore le prime volte: in tal caso far girare nuovamente
output = pd.concat([F,S,G,C,V,M,CI], axis = 0).T #enormi problemi di connessione, quindi ho dovuto dividere in singole tasks
tot = output.T
tot

,IGNORANZA EVERYWHERE,MAINZ NA GIOIA,XYZ,AS 800A,PALLA PAZZA,I DISEREDATI,SOROS FC,PDG 1908
Fantapunti Fatti,64.0,69.0,64.00,71.25,72.75,75.50,67.50,72.75
Fantapunti Subiti,69.0,64.0,71.25,64.00,75.50,72.75,72.75,67.50
Goal subiti,1.0,2.0,2.00,0.00,0.00,0.00,0.00,0.00
C. gialli,0.0,3.0,2.00,1.00,1.00,0.00,0.00,0.00
C. rossi,1.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00
Bonus Panchinari,0.5,0.0,0.00,1.50,0.00,1.50,0.00,0.00
Modificatore,0.0,0.5,0.50,0.00,1.50,1.50,0.00,1.50
tot Infortunati,4.0,4.0,1.00,4.00,1.00,4.00,2.00,5.00


In [75]:
giornata=35
tot = IGNOBEL_tot(giornata)
tot

In [5]:
tot.to_pickle("Dati_storici/Giornata_"+str(giornata)+".pkl")
#crea 8 dataframe individuali con i dati aggiornati fino alla giornata indicata
aggiorna_database(giornata)

Dati aggiornati fino alla 35 giornata


In [69]:
G = pd.read_pickle("Dati_storici/Giornata_22.pkl")
G

,PDG 1908,AS 800A,SOROS FC,IGNORANZA EVERYWHERE,PALLA PAZZA,MAINZ NA GIOIA,XYZ,I DISEREDATI
Fantapunti Fatti,72.00,72.25,65.75,71.75,68.50,71.75,75.75,73.25
Fantapunti Subiti,72.25,72.00,71.75,65.75,71.75,68.50,73.25,75.75
Goal subiti,2.00,1.00,3.00,0.00,0.00,1.00,0.00,1.00
C. gialli,1.00,1.00,0.00,1.00,0.00,1.00,1.00,1.00
C. rossi,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Bonus Panchinari,0.00,2.25,3.50,0.00,1.50,1.00,0.00,3.00
Modificatore,0.50,0.00,0.00,1.50,1.00,0.50,1.50,1.00
tot Infortunati,4.00,4.00,2.00,4.00,1.00,4.00,2.00,4.00


In [55]:
def df_to_dict(df):
    as_list=df.index.tolist()
    idxG = as_list.index('C. gialli')
    idxR = as_list.index('C. rossi')
    as_list[idxG] = 'C_gialli'
    as_list[idxR] = 'C_rossi'
    df.index=as_list
    temp = dict(df)
    temp_2 = {}
    for key, element in temp.items():
        temp_2[key] = dict(element)
    return temp_2

In [56]:
def get_matchday_dict(k):
    files = glob.glob('Dati_storici/*.pkl')
    dict_matchday={}
    for file in files:
        num = re.findall('_\d+.',file)[0][1:-1] 
        dict_matchday[int(num)]=file
    df=pd.read_pickle(dict_matchday[k])
    return df_to_dict(df)

In [70]:
def create_results_post(season, matchday, df, post_to_mongo=False):
    post = {'_id':'2020-21_G'+f'{matchday:02}',
     'matchday':matchday,
     'season':season,
     'results': df_to_dict(df)
    }
    if post_to_mongo:
        coll_res = db['Results']
    return post

In [5]:
#voti_all = scarica_voti(giornata = 3, stagione = '2020-21')
#scarica il dataframe con tutti i voti direttamente da fantacalcio.it

In [72]:
storico_individuale(nome = 'luca', giornata = 35) 
#crea un dataframe individuale con le informazioni utili per gli IGNOBEL, con i dati fino alla giornata scelta
#i dati vengono pescati dai pkl contenuti nella cartella Dati_storici

,gg,pf,ps,gs,c,pan,mod,inf,nome
1,1,72.25,67.5,0,6,0,0,7,luca
2,2,77,64.25,2,1,0,0,5,luca
3,3,73.5,70.5,1,2,1.75,0,3,luca
4,4,65,65,0,2,3,0.5,4,luca
5,5,66,67.25,2,2,0,0,4,luca
6,6,70.25,71,1,2,1.5,0.5,5,luca
7,7,69,67.25,1,1,1.5,0,4,luca
8,8,69.75,73.75,1,2,0.5,0,3,luca
9,9,66.5,62.5,2,4,1,0,2,luca
10,10,67.25,70.75,1,1,0,1,2,luca


In [2]:
#include svincolati
#ST = stats_by_team_full(primavera = True)
ST_INFO = stats_by_team_NO_INFO(stagione = '2021-22',primavera = True) #NO PERSONAL DATA
missing_pl = info_missing_players(collection, ST_INFO) #missing players from our db

100% (7 of 7) |##########################| Elapsed Time: 0:00:09 Time:  0:00:09


In [4]:
missing_pl = info_missing_players(collection, ST_INFO) #missing players from our d

100% (2 of 2) |##########################| Elapsed Time: 0:00:03 Time:  0:00:03


In [40]:
"""database=ST_INFO
missing_ids = find_missing_players(collection, database)
all_pl = []
k=0
while k < len(missing_ids):
    idx = missing_ids[k]
    try:
        infos = personal_info(idx, database)
    except:
        k = k-1
        continue
    dic = {}
    dic['Id'] = idx
    dic['Classe'] = infos[0]
    dic['Eta\''] = infos[1]
    dic['Nome Completo'] = infos[2]
    dic['Nazionalita\''] = infos[3]
    all_pl.append(dic)
    print(k/len(missing_ids))
    k+=1
missing_pl=pd.DataFrame(all_pl, index = missing_ids)"""

"database=ST_INFO\nmissing_ids = find_missing_players(collection, database)\nall_pl = []\nk=0\nwhile k < len(missing_ids):\n    idx = missing_ids[k]\n    try:\n        infos = personal_info(idx, database)\n    except:\n        k = k-1\n        continue\n    dic = {}\n    dic['Id'] = idx\n    dic['Classe'] = infos[0]\n    dic['Eta''] = infos[1]\n    dic['Nome Completo'] = infos[2]\n    dic['Nazionalita''] = infos[3]\n    all_pl.append(dic)\n    print(k/len(missing_ids))\n    k+=1\nmissing_pl=pd.DataFrame(all_pl, index = missing_ids)"

In [5]:
missing_pl

,Id,Classe,Eta',Nome Completo,Nazionalita'
4268,4268,19/04/1989,32,Arnautovic Marko,"Austria, Serbia"
5501,5501,15/05/1999,22,Sigurdsson Arnor,Islanda


In [3]:
ST_INFO.rename(columns = {'Qt. A': 'Qt_A','Qt. I': 'Qt_I'}, inplace=True)
ST_INFO=ST_INFO.drop(columns='Diff.')
ST_INFO.to_pickle("Statistiche_giocatori.pkl")
missing_pl.to_pickle('Missing_Players.pkl')
ST_INFO

,Id,R,Nome,Squadra,Pg,Mv,Mf,Gf,Gs,Rp,...,R+,R-,Ass,Amm,Esp,Au,Qt_A,Qt_I,Nome Squadra,Allenatore
2531,2531,A,LUKAKU,Inter,0,0,0,0,0,0,...,0,0,0,0,0,0,44,44,SOROS FC,musci8
2610,2610,A,RONALDO,Juventus,0,0,0,0,0,0,...,0,0,0,0,0,0,44,44,AS 800A,enzo
507,507,A,MURIEL,Atalanta,0,0,0,0,0,0,...,0,0,0,0,0,0,39,39,PDG 1908,pietro
785,785,A,IMMOBILE,Lazio,0,0,0,0,0,0,...,0,0,0,0,0,0,38,38,PALLA PAZZA,nanni
2841,2841,A,VLAHOVIC,Fiorentina,0,0,0,0,0,0,...,0,0,0,0,0,0,34,34,SOROS FC,musci8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4936,4936,A,MANZARI,Sassuolo,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,svincolato,None
5378,5378,A,ODDEI,Sassuolo,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,svincolato,None
5477,5477,A,WARMING,Torino,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,svincolato,None
5493,5493,A,BOCALON,Venezia,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,svincolato,None


In [5]:
df=pd.read_pickle('Missing_Players.pkl')
df

,Id,Classe,Eta',Nome Completo,Nazionalita'
5507,5507,24/05/2002,19,Busio Gianluca,"Stati Uniti, Italia"
5505,5505,24/01/2002,19,Kaio Jorge,Brasile
5119,5119,24/02/2002,19,Samardzic Lazar,"Germania, Serbia"
5504,5504,10/04/1996,25,Coulibaly Lassana,Mali
5502,5502,13/05/1997,24,Adamonis Marius,Lituania
1947,1947,13/01/1990,31,Fiorillo Vincenzo,Italia
5503,5503,05/11/1995,26,Kechrida Wajdi,"Tunisia, Francia"
